In [8]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
#from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

In [2]:
raw = pd.read_csv('nrao_projects.csv')
raw.head()

,project_code,project_title,project_abstract,fs_type,target
0,2018.1.01205.L,Fifty AU STudy of the chemistry in the disk/en...,The huge variety of planetary systems discover...,line,1
1,2022.1.00316.L,COMPASS: Complex Organic Molecules in Protosta...,The emergence of complex organic molecules in ...,line,1
2,2017.1.00161.L,ALCHEMI: the ALMA Comprehensive High-resolutio...,A great variety in gas composition is observed...,line,1
3,2021.1.01616.L,ALMA JELLY - Survey of Nearby Jellyfish and Ra...,We propose the first ever statistical survey o...,line,1
4,2021.1.00869.L,Bulge symmetry or not? The hidden dynamics of ...,A radio survey of red giant SiO sources in the...,line,1


In [3]:
raw['text'] = raw.project_title.str.strip() + ' ' + raw.project_abstract.str.strip()    # strip leading or trailing spaces
raw['text'] = raw.text.str.lower()                                                      # convert to lower case

In [4]:
raw = raw.drop(['project_code', 'project_title', 'project_abstract', 'fs_type'], axis=1)

In [5]:
NGRAM_RANGE = (1, 10)
TOP_K = 20000
TOKEN_MODE = 'word'
MIN_DOC_FREQ = 1

tfidf_vectorizer = TfidfVectorizer()
x_train = tfidf_vectorizer.fit_transform(raw['text'].astype('U'))

In [6]:
test = pd.DataFrame(tfidf_vectorizer.get_feature_names_out())

In [7]:
test[1989:]

,0
1989,abandance
1990,abandunce
1991,abcs
1992,abell
1993,aberrant
...,...
14530,zyjhk
14531,µjy
14532,µm
14533,λcdm


In [ ]:
#convert to lowercase, strip and remove punctuations
def preprocess(text):
    text = text.lower()
    text=text.strip()  
    text=re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 
    return text
 
# STOPWORD REMOVAL
def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)

#LEMMATIZATION
# Initialize the lemmatizer
wl = WordNetLemmatizer()
 
# This is a helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN# Tokenize the sentence
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)

# Notes
## Text Standardization
* Respect and build thoughtful OHCO for overall text model

### Tentative Language Model Deliverable
* LIB table (collection of proposal IDs, titles, abstracts, title+abstract, possible classification but maybe this could be reseved for a different table)
* DOC table (gather from token table)
* TOKEN table ()
* VOCAB table ()

### OHCO Outline
* Combine title and abstract
* Corpus is built of combined titles and abstracts, i.e. all proposal titles and abstracts
* Treat combined title and abstract as document
* **CURRENTLY IT APPEARS AS THOUGH ABSTRACTS ARE FORMATTED TO ONE LARGE PARAGRAPH THOUGH THIS REQUIRES EDA CONFIRMATION**
* **IT APPEARS AS THOUGH TITLES ARE ONE SENTENCE BUT THIS REQURES EDA CONFIRMATION**
* Combine title and abstract on specific (unique) character to ensure full title is first sentence of document
* THIS GIVES THE FOLLOWING OHCO
    * CORPUS (all projects)
    * DOCUMENT (individual project, title and abstract combined)
    * SENTENCE (title is first sentence, abstract are following sentences)
    * TOKEN (see notes below on tokenization)

### Tokenization notes
* What to do with numbers?
* Lower case everything
* Strip leading and trailing space